In [12]:
import sys, pathlib, importlib

proj_root = pathlib.Path("..").resolve()
if str(proj_root) not in sys.path:
    sys.path.insert(0, str(proj_root))

from src.data_loading import load_eurusd
from src.features import add_features, FEATURE_COLS, TARGET_COL
from src.modeling import (
    train_test_split_time,
    build_logreg_pipeline,
    build_decision_tree,
    build_random_forest,
    build_gradient_boosting,
)

from src.evaluation import evaluate_classifier
import pandas as pd

df = load_eurusd()
df_feat = add_features(df)

X_train, X_test, y_train, y_test = train_test_split_time(df_feat, test_size=0.2)


In [13]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train)
pred = dummy.predict(X_test)

print("Dummy accuracy:", round(accuracy_score(y_test, pred), 3))

dummy_s = DummyClassifier(strategy="stratified", random_state=0)
dummy_s.fit(X_train, y_train)
proba = dummy_s.predict_proba(X_test)[:, 1]
print("Dummy ROC-AUC:", round(roc_auc_score(y_test, proba), 3))


Dummy accuracy: 0.499
Dummy ROC-AUC: 0.494


In [14]:
models = {
    "LogReg": build_logreg_pipeline(),
    "DecisionTree": build_decision_tree(),
    "RandomForest": build_random_forest(),
    "GradientBoosting": build_gradient_boosting(),
}

results = []

for name, m in models.items():
    m.fit(X_train, y_train)
    acc, auc = evaluate_classifier(m, X_train, y_train, X_test, y_test, name=name)
    results.append((name, acc, auc))



=== LogReg ===
Accuracy: 0.49
ROC-AUC : 0.483
Confusion matrix:
 [[689 711]
 [720 685]]

Classification report:
               precision    recall  f1-score   support

           0      0.489     0.492     0.491      1400
           1      0.491     0.488     0.489      1405

    accuracy                          0.490      2805
   macro avg      0.490     0.490     0.490      2805
weighted avg      0.490     0.490     0.490      2805

=== DecisionTree ===
Accuracy: 0.494
ROC-AUC : 0.497
Confusion matrix:
 [[583 817]
 [602 803]]

Classification report:
               precision    recall  f1-score   support

           0      0.492     0.416     0.451      1400
           1      0.496     0.572     0.531      1405

    accuracy                          0.494      2805
   macro avg      0.494     0.494     0.491      2805
weighted avg      0.494     0.494     0.491      2805

=== RandomForest ===
Accuracy: 0.487
ROC-AUC : 0.481
Confusion matrix:
 [[749 651]
 [789 616]]

Classification re

In [10]:
from src.modeling import grid_search_timeseries
from sklearn.ensemble import RandomForestClassifier

rf_base = RandomForestClassifier(random_state=0, n_jobs=-1)

param_grid_rf = {
    "n_estimators": [200, 400],
    "max_depth": [4, 6, 8],
    "min_samples_leaf": [20, 50],
}

rf_best, rf_params, rf_cvscore = grid_search_timeseries(
    rf_base, param_grid_rf, X_train, y_train, n_splits=5
)

print("Best RF params:", rf_params)
print("CV ROC-AUC:", round(rf_cvscore, 3))

acc_rf_best, auc_rf_best = evaluate_classifier(
    rf_best, X_train, y_train, X_test, y_test, name="RandomForest tuned"
)


Best RF params: {'max_depth': 6, 'min_samples_leaf': 20, 'n_estimators': 200}
CV ROC-AUC: 0.509
=== RandomForest tuned ===
Accuracy: 0.484
ROC-AUC : 0.479
Confusion matrix:
 [[769 631]
 [816 589]]

Classification report:
               precision    recall  f1-score   support

           0      0.485     0.549     0.515      1400
           1      0.483     0.419     0.449      1405

    accuracy                          0.484      2805
   macro avg      0.484     0.484     0.482      2805
weighted avg      0.484     0.484     0.482      2805



In [11]:
results.append(("RandomForest tuned", acc_rf_best, auc_rf_best))

pd.DataFrame(results, columns=["model", "accuracy", "roc_auc"]).sort_values("roc_auc", ascending=False)


,model,accuracy,roc_auc
1,DecisionTree,0.494118,0.497120
3,GradientBoosting,0.493048,0.485802
0,LogReg,0.489840,0.483170
2,RandomForest,0.486631,0.481261
4,RandomForest tuned,0.484135,0.478820
